# cm967's Spectra Calculator
author: Camillo Moschner | start: December 2020 | last modified: 06.01.2023 (polished & uploaded to thesis GitHub)

## Import Statements

In [1]:
import numpy as np
import pandas as pd
from ipywidgets import interact
import seaborn as sns
import os

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.patches as patches

/Users/camillomoschner/opt/anaconda3/envs/ia-env/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Function Definitions

In [2]:
def wavelength_to_rgb(wavelength, gamma=0.8):
    ''' taken from http://www.noah.org/wiki/Wavelength_to_RGB_in_Python
    This converts a given wavelength of light to an approximate RGB color value. The wavelength must be given
    in nanometers in the range from 380 nm to 750 nm(789 THz through 400 THz). Additionally alpha value set to 
    0.5 outside range. Based on code by Dan Bruton http://www.physics.sfasu.edu/astro/color/spectra.html
    '''
    wavelength = float(wavelength)
    if wavelength >= 380 and wavelength <= 750:
        A = 1.
    else:
        A=0.2
    if wavelength < 380:
        wavelength = 380.
    if wavelength >750:
        wavelength = 750.
    if wavelength >= 380 and wavelength <= 440:
        attenuation = 0.3 + 0.7 * (wavelength - 380) / (440 - 380)
        R = ((-(wavelength - 440) / (440 - 380)) * attenuation) ** gamma
        G, B = 0.0, (1.0 * attenuation) ** gamma
    elif wavelength >= 440 and wavelength <= 490:
        R = 0.0
        G = ((wavelength - 440) / (490 - 440)) ** gamma
        B = 1.0
    elif wavelength >= 490 and wavelength <= 510:
        R, G = 0.0, 1.0
        B = (-(wavelength - 510) / (510 - 490)) ** gamma
    elif wavelength >= 510 and wavelength <= 580:
        R = ((wavelength - 510) / (580 - 510)) ** gamma
        G, B = 1.0, 0.0
    elif wavelength >= 580 and wavelength <= 645:
        R = 1.0
        G = (-(wavelength - 645) / (645 - 580)) ** gamma
        B = 0.0
    elif wavelength >= 645 and wavelength <= 750:
        attenuation = 0.3 + 0.7 * (750 - wavelength) / (750 - 645)
        R = (1.0 * attenuation) ** gamma
        G, B = 0.0, 0.0
    else:
        R, G, B = 0.0, 0.0, 0.0
    return (R,G,B,A)

def plot_spectrum(axis, df, column_name, f_type=''):
    if ('em' in column_name) or ('EM' in column_name):
        line = '-'
    elif ('ex' in column_name) or ('EX' in column_name):
        line = '--'
    fluorophore_data = df[column_name]
    fluorophore_data = fluorophore_data.loc[pd.notna(fluorophore_data)]
    axis.plot(fluorophore_data.index, fluorophore_data, linestyle=line, 
              color=wavelength_to_rgb(df.index[df[column_name] == df[column_name].max()].tolist()[0],gamma=1.8),
              label = f"{column_name} ({f_type})")

# Load Files

In [3]:
fp_spectra_df = pd.read_csv(f"spectra_data{os.path.sep}all_fp_spectra.csv",index_col=0)
print(f"fp_spectra_df contains data located in a range of {len(fp_spectra_df)} nm (visible spectrum of light = 500 nm).")
avail_fluorophore_spectra_df = pd.read_csv(f"spectra_data{os.path.sep}avail_fluorophore_spectra.csv",index_col=0)
avail_nuclear_dye_spectra_df = pd.read_csv(f"spectra_data{os.path.sep}nuclear_dye_spectra.csv",index_col=0)
# LEDs
LED_list = [365,440,488, 514,561,594, 640,730]
all_LED_data_df = pd.read_csv(f"spectra_data{os.path.sep}all_LED_data_SynBioLab.csv",index_col=0)
# Dichroic Cubes
CFP_YFP_mCherry_A_filter_cube_data = pd.read_csv(f"spectra_data{os.path.sep}LED-CFP_YFP_mCherry-A-000 Filter Cube (32mm).csv",index_col=0)
DA_FI_TR_Cy5_B_filter_cube_data = pd.read_csv(f"spectra_data{os.path.sep}LED-DA_FI_TR_Cy5-B Filter Cube (32mm).csv",index_col=0)
dichroic_dic = {'CFP_YFP_mCherry': CFP_YFP_mCherry_A_filter_cube_data, 
                'DA_FI_TR_Cy5_B' : DA_FI_TR_Cy5_B_filter_cube_data}
# Emission Filters
em_filter_list = [{'center':435,'width':26}, {'center':475,'width':20},
                  {'center':515,'width':30}, {'center':540,'width':21},
                  {'center':595,'width':40}, {'center':632,'width':60},
                  {'center':705,'width':72}]            

fp_spectra_df contains data located in a range of 1400 nm (visible spectrum of light = 500 nm).


# Plot w/ Lists

In [4]:
fp_list = [ # 'mVenus','Superfolder GFP', 'mCerulean'
          ]
chem_fluorophore_list = ['Cy3','Cy5' # '6-FAM'
                         ]
nuclear_dye_list = [] # 'DAPI'
all_fluorophores = fp_list + chem_fluorophore_list + nuclear_dye_list
all_fluorophores_df = pd.concat([fp_spectra_df, avail_fluorophore_spectra_df, avail_nuclear_dye_spectra_df])
# Quick check that you have data for all fluorophores you want to investigate
result = any( [ (f"{fluo} ex" in all_fluorophores_df.columns) for fluo in all_fluorophores] )
print(f" -> Is data available for all fluorophores to investigate? : { result }")

 -> Is data available for all fluorophores to investigate? : True


In [5]:
@interact 
def show_plot(LED_no =(0,len(LED_list)-1),
              dichroic = dichroic_dic.keys(),
              all_EM = [True, False],
              EM_filter = em_filter_list
              ):
    plt.style.use('default')
    fig = plt.figure(figsize=(14,9))
    gs = gridspec.GridSpec(2,1,figure=fig)
    ax_fluo, ax_dichroic = fig.add_subplot(gs[0,0]), fig.add_subplot(gs[1,0])
    fig.suptitle("\nSpectra Viewer by Camillo Moschner\n",size=14)
    fluorophore_names = ' - '.join([', '.join(fluo) for fluo in [fp_list, chem_fluorophore_list, nuclear_dye_list]])
    # Fluorophores
    if len(fp_list) != 0:
        [(plot_spectrum(ax_fluo, fp_spectra_df, f"{fluo} ex", f_type='FP'), plot_spectrum(ax_fluo, fp_spectra_df, f"{fluo} em", f_type='FP')) for fluo in fp_list]
    if len(chem_fluorophore_list) != 0:
        [(plot_spectrum(ax_fluo, avail_fluorophore_spectra_df, f"{fluo} ex", f_type='s.c.'), plot_spectrum(ax_fluo, avail_fluorophore_spectra_df, f"{fluo} em", f_type='s.c.')) for fluo in chem_fluorophore_list]
    if len(nuclear_dye_list) != 0:
        [(plot_spectrum(ax_fluo, avail_nuclear_dye_spectra_df, f"{fluo} EX", f_type='s.c.'), plot_spectrum(ax_fluo, avail_nuclear_dye_spectra_df, f"{fluo} EM", f_type='s.c.')) for fluo in nuclear_dye_list]
    # LEDs
    [ax_fluo.vlines(LED_lambda, 0,1,linewidth=1.5,color= wavelength_to_rgb(LED_lambda, gamma=1.5)) for LED_lambda in LED_list]
    [ax_fluo.vlines(wavelength, 1.05,1.14,linewidth=3,color= wavelength_to_rgb(wavelength, gamma=0.8)) for wavelength in range(300,800)]
    ax_fluo.vlines(LED_list[LED_no], 0,1,linewidth=4.5,color= wavelength_to_rgb(LED_list[LED_no], gamma=0.9))
    # Dichroic
    [ax_dichroic.plot(dichroic_dic[dichroic][spectrum].index, dichroic_dic[dichroic][spectrum], 
                     '-', color = c_colour, alpha=0.5,linewidth=2, markersize=12,label=str(spectrum)) 
     for (spectrum,c_colour) in [('excitation','red'), ('dichroic','green'),('emission','blue')]]
    # Emission Filters
    if all_EM:
        for em_data in em_filter_list:
            em_filter_start = em_data['center']-em_data['width']/2
            em_filter_end = em_data['center']+em_data['width']/2
            plt.rcParams["hatch.linewidth"] = 4
            em_filter_patch = patches.Rectangle((em_filter_start, 0),em_data['width'], 1, 
                                                linewidth=1, edgecolor='black', alpha = 0.5,label=f"{em_data['center']} EM filter",
                                                facecolor=wavelength_to_rgb(em_data['center']),hatch=r"//")
            ax_fluo.add_patch(em_filter_patch)
    else:
        em_filter_start = EM_filter['center']-EM_filter['width']/2
        em_filter_end = EM_filter['center']+EM_filter['width']/2
        plt.rcParams["hatch.linewidth"] = 4
        em_filter_patch = patches.Rectangle((em_filter_start, 0),EM_filter['width'], 1, 
                                            linewidth=1, edgecolor='black', alpha = 0.5,label=f"{EM_filter['center']} EM filter",
                                            facecolor=wavelength_to_rgb(EM_filter['center']),hatch=r"//")
        ax_fluo.add_patch(em_filter_patch)
    # Nikon-given Lumencore LED spectra:
    for column_name in all_LED_data_df.columns:
        current_LED_data = all_LED_data_df.loc[pd.notna(all_LED_data_df[column_name])][column_name]
        normalised_LED_data = current_LED_data/all_LED_data_df.describe().loc['max'].max()
        # sns.lineplot(current_LED_data.index, 
        #              normalised_LED_data,
        #              ax=ax_dichroic,label=column_name, 
        #              linewidth=1,color= wavelength_to_rgb(current_LED_data.idxmax(), gamma=0.8))
        plt.fill_between(current_LED_data.index, normalised_LED_data,color= wavelength_to_rgb(current_LED_data.idxmax(), gamma=0.4))

    ax_fluo.legend(loc = 'center right'), ax_dichroic.legend(loc = 'center right')
    # subplot modification
    ax_fluo.set_title(f"Fluorophores: {fluorophore_names}\n| LED: {LED_list[LED_no]} | Dichroic: {dichroic} | EM Filter: {EM_filter} | ",size=12)
    #ax_fluo.set_ylabel('Intensity'), ax_dichroic.set_ylabel('Transmittance')
    [(subplot.set_ylim(0,1.15),subplot.set_xlim(300,801),
      subplot.set_xticks(np.arange(300, 800+1, 25)), #subplot.set_xlabel('Wavelength (nm)'),
      subplot.spines['right'].set_visible(False), subplot.spines['top'].set_visible(False),
      subplot.grid(color='grey', linestyle='--', linewidth=0.5, alpha=0.25)) for subplot in [ax_fluo,ax_dichroic]]
    ax_dichroic.set_xlabel('Wavelength (nm)')
    #plt.savefig(f"cam_spectra_viewer.pdf", dpi=300)
    plt.show(); 
    

interactive(children=(IntSlider(value=3, description='LED_no', max=7), Dropdown(description='dichroic', option…